<a href="https://colab.research.google.com/github/hugomaharavo/ai4eng-20242-Pruebas-Saber-Pro-Colombia/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1 - ACCEDER AL ARCHIVO TRAIN.CSV
# 1.1 - Instale la biblioteca opendatasets para descargar conjuntos de datos directamente de Kaggle
!pip install opendatasets

# 1.2 - Importar los módulos necesarios
import opendatasets as kaggle_data
import os

# 1.3 - Enlace al concurso de Kaggle
competition_url = "https://www.kaggle.com/competitions/udea-ai4eng-20242"

# 1.4 - Descargar los datos del concurso
kaggle_data.download(competition_url)

# 1.5 - Cambiar de directorio para acceder a la carpeta del conjunto de datos descargado
project_dir = "udea-ai4eng-20242"
os.chdir(project_dir)

# 1.6 - Comprobación de los archivos descargados
files = os.listdir()
print("Fichiers disponibles dans le dossier : ", files)


# 2 - Importar las bibliotecas necesarias
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# 3 - Carga de datos
train_df = pd.read_csv("train.csv")


# 4 - CONVERSIÓN DE VALORES ORDINALES EN VALORES NUMÉRICOS PROMEDIO

# 4.1 - Para ESTU_VALORMATRICULAUNIVERSIDAD: Los valores se basan en estimaciones promedio en millones de pesos
tuition_average_mapping = {
    "No pagó matrícula": 0.0,
    "Menos de 500 mil": 0.25,
    "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75,
    "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millones y menos de 5.5 millones": 4.75,
    "Entre 5.5 millones y menos de 7 millones": 6.25,
    "Más de 7 millones": 7.5
}
train_df['ESTU_VALORMATRICULAUNIVERSIDAD'] = train_df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)

# 4.2 - Para FAMI_ESTRATOVIVIENDA: Los estratos se mantienen como enteros, pero «Sin Estrato» sigue siendo un valor perdido
housing_strata_mapping_average = {
    "Estrato 1": 1,
    "Estrato 2": 2,
    "Estrato 3": 3,
    "Estrato 4": 4,
    "Estrato 5": 5,
    "Estrato 6": 6,
    "Sin Estrato": None  # Sin Estrato sigue siendo el valor que falta
}
train_df['FAMI_ESTRATOVIVIENDA'] = train_df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping_average)

# 4.3 - Para FAMI_EDUCACIONPADRE
education_father_mapping = {
    "No aplica": None,
    "No sabe": None,
    "Ninguno": 0,
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Educación profesional incompleta": 5,
    "Educación profesional completa": 6,
    "Técnica o tecnológica incompleta": 7,
    "Técnica o tecnológica completa": 8,
    "Postgrado": 9
}
train_df['FAMI_EDUCACIONPADRE'] = train_df['FAMI_EDUCACIONPADRE'].map(education_father_mapping)

# 4.4 - Para FAMI_EDUCACIONMADRE
education_mother_mapping = {
    "No aplica": None,
    "No sabe": None,
    "Ninguno": 0,
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Educación profesional incompleta": 5,
    "Educación profesional completa": 6,
    "Técnica o tecnológica incompleta": 7,
    "Técnica o tecnológica completa": 8,
    "Postgrado": 9
}
train_df['FAMI_EDUCACIONMADRE'] = train_df['FAMI_EDUCACIONMADRE'].map(education_mother_mapping)


# 5 - CODIFICACIÓN BINARIA

# 5.1 - Para la columna FAMI_TIENEINTERNET
internet_mapping = {
    "Si": 1,
    "No": 0
}
train_df['FAMI_TIENEINTERNET'] = train_df['FAMI_TIENEINTERNET'].map(internet_mapping)


# 5.2 - Para la columna ESTU_PAGOMATRICULAPROPIO
tuition_payment_mapping = {
    "Si": 1,
    "No": 0
}
train_df['ESTU_PAGOMATRICULAPROPIO'] = train_df['ESTU_PAGOMATRICULAPROPIO'].map(tuition_payment_mapping)


# 6 - Codificación de la variable objetivo
target_order = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
train_df['RENDIMIENTO_GLOBAL'] = train_df['RENDIMIENTO_GLOBAL'].map(target_order)


# 7 - Separación de características y objetivo
X = train_df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y = train_df['RENDIMIENTO_GLOBAL']


# 8 - Identificación de columnas para pretratamiento
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns


# 9 - Creación de un proceso de transformación
preprocessor = ColumnTransformer(
    transformers=[
        # Imputación de valores numéricos con la mediana
        ('num', SimpleImputer(strategy='median'), numeric_columns),

        # Imputación y codificación OneHot para variables categóricas seleccionadas
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_columns)
    ],
    remainder='passthrough'  # Conserva las columnas ya procesadas manualmente
)

X_cleaned = preprocessor.fit_transform(X) # Aplicación del proceso de preprocesamiento a los datos de entrenamiento

# 10 - Conversión del resultado en un DataFrame para su verificación.
X_cleaned_df = pd.DataFrame(X_cleaned)

# 11 - Guardar y descargar datos preprocesados
train_cleaned_df = pd.concat([X_cleaned_df, y.reset_index(drop=True)], axis=1)
train_cleaned_df.to_csv("train_cleaned.csv", index=False)

from google.colab import files
files.download('train_cleaned.csv')